In [36]:
from warnings  import  filterwarnings
filterwarnings('ignore')

In [37]:
import  pandas as pd
import  seaborn as  sns
import  numpy as  np
import matplotlib.pyplot as  plt

In [38]:
df  =  pd.read_csv(r"C:\Users\admin\Downloads\insuranceLiner.csv")

### basic quality chceks

In [40]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [41]:
df.shape

(1338, 7)

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [43]:
df.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [44]:
df.duplicated().sum()

np.int64(1)

In [45]:
df.drop_duplicates(keep = 'first' ,  inplace  =  True ,  ignore_index= True)

In [46]:
df.shape

(1337, 7)

# seperate x &y

In [47]:
df.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges'], dtype='object')

In [48]:
x =  df.drop('charges' , axis= 1)
y  =  df['charges']

### seperate categorical & numerical  data

In [49]:
cat  =   x.columns[x.dtypes  =='object']
num  = x.columns[x.dtypes  !='object']

In [50]:
print(cat)
print()
print()
print(num)

Index(['sex', 'smoker', 'region'], dtype='object')


Index(['age', 'bmi', 'children'], dtype='object')


# create pipeline

In [51]:
from sklearn.pipeline  import  Pipeline
from sklearn.preprocessing import  StandardScaler ,  OneHotEncoder
from  sklearn.impute  import  SimpleImputer
from sklearn.compose  import ColumnTransformer

In [52]:
num_pipeline  =  Pipeline(steps =  [('imputer' , SimpleImputer(strategy= 'mean')) ,
                                      ('scaler' ,  StandardScaler())])

cat_pipeline  =  Pipeline(steps  =  [('imputer' ,  SimpleImputer(strategy='most_frequent' ) ) ,
                                       ('encoder' ,  OneHotEncoder(handle_unknown='ignore'))])

In [53]:
# create  pre
pre =  ColumnTransformer([('num' ,  num_pipeline , num) , 
                           ('cat' ,  cat_pipeline ,  cat)])

In [54]:
pre

,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'mean'
,fill_value,None


In [55]:
x_scaled  =  pre.fit_transform(x)

In [56]:
x_scaled = pd.DataFrame(x_scaled ,  columns  =  pre.get_feature_names_out())

In [57]:
x_scaled.head()

,num__age,num__bmi,num__children,cat__sex_female,cat__sex_male,cat__smoker_no,cat__smoker_yes,cat__region_northeast,cat__region_northwest,cat__region_southeast,cat__region_southwest
0,-1.440418,-0.453160,-0.909234,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,-1.511647,0.509422,-0.079442,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
2,-0.799350,0.383155,1.580143,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,-0.443201,-1.305052,-0.909234,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
4,-0.514431,-0.292456,-0.909234,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0


### train  test split

In [58]:
from sklearn.model_selection  import  train_test_split

In [59]:
x_train  , x_test ,  y_train  , y_test  =   train_test_split(x_scaled  , y  ,  random_state=23  ,  test_size  = 0.2)

In [60]:
print(x_train.shape  ,  y_train.shape)
print(x_test.shape  , y_test.shape)

(1069, 11) (1069,)
(268, 11) (268,)


### model  fitting

In [61]:
from sklearn.neighbors import  KNeighborsRegressor ,  KNeighborsClassifier

In [62]:
model  =  KNeighborsRegressor(n_neighbors = 3 )


In [63]:
model.fit(x_train ,  y_train)

,n_neighbors,3
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


In [64]:
model.score(x_train  , y_train)

0.8921870067923343

In [65]:
model.score(x_test , y_test)

0.7143987033726368

### hyper parameter tunning

In [66]:
from sklearn.model_selection import  GridSearchCV

In [67]:
para  =  {'n_neighbors' : range(2 , 10) ,  
          'metric' : ['euclidean' ,  'manhattan' , 'minkowski' ]}
KNN  = KNeighborsRegressor()

In [68]:
gscv =  GridSearchCV(KNN  ,  
                     param_grid= para,
                         n_jobs= -1  ,  
                   
                         verbose= 1)

In [69]:
gscv

,estimator,KNeighborsRegressor()
,param_grid,"{'metric': ['euclidean', 'manhattan', ...], 'n_neighbors': range(2, 10)}"
,scoring,None
,n_jobs,-1
,refit,True
,cv,None
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,n_neighbors,5


In [70]:
gscv.fit( x_train  ,  y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


,estimator,KNeighborsRegressor()
,param_grid,"{'metric': ['euclidean', 'manhattan', ...], 'n_neighbors': range(2, 10)}"
,scoring,None
,n_jobs,-1
,refit,True
,cv,None
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,n_neighbors,5


In [71]:
gscv.best_params_

{'metric': 'manhattan', 'n_neighbors': 5}

In [72]:
gscv.score(x_train , y_train)

0.8608312283442743

In [73]:
gscv.score(x_test  , y_test)

0.7396081936316048

In [75]:
from  sklearn.metrics  import  (mean_squared_error ,
                                 root_mean_squared_error , 
                                 mean_absolute_error , 
                                 mean_absolute_percentage_error ,  r2_score)

### Evaluation

In [76]:
def evalaute(model  ,  x, y):
    pred  =  model.predict(x)
    MSE  =  mean_squared_error(y , pred)
    RMSE  = root_mean_squared_error(y , pred)
    MAE  =  mean_absolute_error(y , pred)
    MAPE =  mean_absolute_percentage_error(y , pred)
    r2  =  r2_score(y , pred)

    print(f'MSE -- >  {MSE}')
    print(f'RMSE -- >  {RMSE}')
    print(f'MAE -- >  {MAE}')
    print(f'MAPE -- >  {MAPE}')
    print(f'R2-- >  {r2}')

    

In [77]:
# training evaluation
evalaute(gscv  ,  x_train  ,  y_train)

MSE -- >  20540346.875763454
RMSE -- >  4532.1459459910875
MAE -- >  2690.871963927596
MAPE -- >  0.27830646056220193
R2-- >  0.8608312283442743


In [79]:
# testing evaluation
evalaute(gscv  ,  x_test  ,  y_test)

MSE -- >  37078475.30514473
RMSE -- >  6089.209743894911
MAE -- >  3812.682894193283
MAPE -- >  0.37290726212105607
R2-- >  0.7396081936316048


In [80]:
from  sklearn.linear_model import LinearRegression
LR  =  LinearRegression()

In [81]:
LR.fit(x_train ,  y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [82]:
LR.score(x_train  ,  y_train)

0.7591922785977974

In [83]:
LR.score(x_test  , y_test)

0.7134874121725816

In [84]:
from sklearn.tree import DecisionTreeRegressor

In [85]:
DT  = DecisionTreeRegressor(max_depth  =  5 , min_samples_split= 15)

In [86]:
DT.fit(x_train  ,  y_train)

,criterion,'squared_error'
,splitter,'best'
,max_depth,5
,min_samples_split,15
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,ccp_alpha,0.0


In [87]:
DT.score(x_train  , y_train)

0.8835672676411431

In [88]:
DT.score(x_test  ,  y_test)

0.8492965573439171